# <span style='font-family:Georgia'> Kelly's Principle

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from Stats&Probability


SyntaxError: invalid syntax (<ipython-input-1-ed61d2c48134>, line 5)

- [**Here**](http://www.princeton.edu/~wbialek/rome/refs/kelly_56.pdf) Kelly's paper
- [**Taleb Video**](https://www.youtube.com/watch?v=91IOwS0gf3g&list=WL&index=1&t=300s) to explain it
- [**Thorp Paper**](http://www.eecs.harvard.edu/cs286r/courses/fall12/papers/Thorpe_KellyCriterion2007.pdf)

Consider a fair bet, quoted at 2 (50/50 chance). Let's first assume we know with certainty the winner. How much would
he bet? Probably all he had since you would win with certainty. In this case The capital would grow exponentially and after N bets he would have $2^N$ times the original bankroll. This exponential growth of capital is not uncommon in economics. In fact, if the binary digits in the above channel were arriving at the rate of one per week, the sequence of bets would have the value of an investment paying 100% interest per week compounded weekly. We will make use of a quantity $G$ called the exponential rate of growth of the gambler’s capital, where

$$G = \lim_{N \to +\infty} \frac{1}{N} \log \frac{V_N}{V_0}$$

That means

$$V_N = V_0 e^{G N}$$

In the previous example, $G=1$. In a discrete framework, 

$$V_N = V_0 (1+r)^N$$

and so:

$$r + 1 = \frac{V_N}{V_0}^{\frac{1}{N}}$$

**By setting $G = \log(1+r)$ we obtain the first equation**. Consider the case now of a noisy binary channel, where each transmitted symbol has probability, $p$ of correct transmission. Now the gambler could still bet his entire capital each time, and, in fact, this would maximize the expected value of his capital, which in this case would be given by

$$E(V_N)  = (2p)^N V_0$$

(**this is true only if $p > 0.5$**)


This would be little comfort, however, since when N was large he would probably be broke and, in fact, would be broke with probability one if he continued indefinitely. Let us, instead, assume that he bets a fraction $l$ of his capital each time. Then

$$V_N = (1 + l)^W (1 − l)^L V_0$$

<div class="alert alert-info">
To reach to the previous conclusion that the expected value of the bet is reached as betting the full wealth let's take the expected value of that quantity

$$
E(V_N) = E((1 + l)^W (1 − l)^L V_0)\\
E(V_N) = (1 + l)^{Np} (1 − l)^{N(1-p)} V_0\\
$$


Thorp Paper
</div>

where $W$ and $L$ are the number of wins and losses in the $N$ bets. Then

$$\begin{align} G &= \lim_{N \to +\infty} \frac{1}{N} \log \frac{(1 + l)^W (1 − l)^L V_0}{V_0} \\
&= \lim_{N \to +\infty} \left[ \frac{W}{N} \log(1 + l) + \frac{L}{N} \log(1 - l) \right] \\
&= p \log(1 + l) + (1-p) \log(1 - l) \end{align}$$

Where $p$ is the probability of winning. Maximizing $G$ means deriving last equation and set it equal to zero.

$$\frac{\partial G}{\partial l} = \frac{p}{1+l} - \frac{1-p}{1-l}$$

which is convex since

$$\frac{\partial^2 G}{\partial l^2} = -\frac{p}{(1+l)^2} - \frac{1-p}{(1-l)^2}$$

is always negative. Setting the derivative equal to zero yields:

$$1+l = 2p\\
$$

In [ ]:
bet_portions = np.arange(0.01, 1, 0.001) # waged fraction
correct_transmission_prob = np.arange(0.5, 1.05, 0.05) # chance of correct communication

In [ ]:
q = 0.8
g_dict = {}
for q in correct_transmission_prob:
    q = round(q, 2)
    g_dict_by_prob = {}
    for l in bet_portions:
        g = q * np.log(1 + l) + (1-q) * np.log(1-l)
        g_dict_by_prob[l] = g
    
    g_dict[q] = g_dict_by_prob


In [ ]:
log_G = pd.DataFrame(g_dict)  # logaritmic growth
G = (np.exp(log_G) - 1)  # growth

In [ ]:
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(1, 1, 1)
log_G.plot(ax=ax,  cmap=plt.get_cmap('inferno'))
max_g = log_G.max(axis=0)
max_g.index = log_G.idxmax(axis=0, skipna=True).values
max_g = max_g.reset_index()
max_g.columns =['x', 'y']
max_g.plot(kind ='scatter', x='x', y='y', ax=ax, color = 'red', s=50)
ax.set_ylim(-1.5, 1.1)
ax.set_title('Log Growth', fontsize=20)
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(1, 1, 1)
G.plot(ax=ax, cmap=plt.get_cmap('viridis'))
max_g = G.max(axis=0)
max_g.index = G.idxmax(axis=0, skipna=True).values
max_g = max_g.reset_index()
max_g.columns =['x', 'y']
max_g.plot(kind ='scatter', x='x', y='y', ax=ax, color = 'red', s=50)
ax.set_title('Linear Growth', fontsize=20)
ax.set_ylim(-1.5, 1.1)
plt.show()

## <span style='font-family:Georgia'> Generalizations
    
- In the base case, i.e. doubling the wagered amount with probability $p$ or losing everything with probability $q = 1 - p$, the optimal fraction of wealth wagered is: 

    $$f^* = p - q$$  


- Winning $b$ units per unit of wagered amount with probability $p$ or losing everything with probability $q = 1 - p$, the optimal fraction of wealth wagered is: 

    $$f^* = \frac{bp - q}{b}$$  
    
    Of course, if $b=2$, we go back to the base case.

- Winning $b$ units per unit of wagered amount with probability $p$ or losing $a$ units per unit of wagered amount with probability $q$, then, if the expectation value of the bet $bp-aq > 0$, the optimal fraction of wealth wagered is: 

    $$f^* = \frac{bp - aq}{ab}$$  
    
    Of course, if $b=2$, we go back to the base case.

   

In [ ]:
def optimal_f(p, b=2, a=0):
    
    q = 1 - p
    
    if a > 0:
        opt_f = (b * p - a * q)/(a * b) 
    else:
        opt_f = (b * p - q)/b 
        
    return opt_f

In [ ]:
optimal_f(0.8)

In [ ]:

# 100 linearly spaced numbers
x = np.linspace(-1,1,100)

# the function, which is y = x^3 here
y = x**3

# setting the axes at the centre
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(1, 1, 1)
ax.spines['left'].set_position('center')
ax.spines['bottom'].set_position('center')
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')

# plot the function
plt.plot(x,y, 'g')

# show the plot
plt.show()